In [ ]:
import torch
from torch_geometric.data import Data
import os

data = torch.load(r"dataset/ogbn_mag/processed/geometric_data_processed.pt", map_location='cpu')

# Check the contents
print(data)

/tmp/ipykernel_19046/3940350772.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(r"dataset/ogbn_mag/processed/geometric_data_processed.pt", map_location

FileNotFoundError: [Errno 2] No such file or directory: 'dataset/ogbn_mag/processed/geometric_data_processed.pt'